<a href="https://colab.research.google.com/github/rick1270/Clayton_DS1_Projects/blob/master/Clayton_DS1_Project1_CA_Water.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Project Action Plan
#### Phase One
1.  Develop Hypothesis regarding water quality 
2.  Research parameters, set forth by the EPA determin their relevance and availibility.  Choose aproximately 12 parameters (drop 2 if needed) and number of water plants

3.  Establish time frame and data points for project

#### Phase Two
1.  Download data from state
2.  Review data (shape, discription, headers)
3.  Determin frequency and distribution of missing data
4.  Address missing data

#### Phase Three
1.  Produce  plot consisting of state of California, data collection points, colors indicating water component levels, with two sliders for year and component.
2.  Produce line graph showing five largest cities in CA and change in each component over time frame


In [323]:
!apt install proj-bin libproj-dev libgeos-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libgeos-dev is already the newest version (3.6.2-1build2).
libproj-dev is already the newest version (4.9.3-2).
proj-bin is already the newest version (4.9.3-2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [324]:
!pip install https://github.com/matplotlib/basemap/archive/v1.1.0.tar.gz

     | 165.5MB 18.0MB/s
  Running setup.py bdist_wheel for basemap ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / done
  Stored in directory: /tmp/pip-ephem-wheel-cache-e7vp58ja/wheels/6c/c2/bd/d6fb8f2660e0591d357352aba1b117b3211d016994d933d792
Successfully built basemap


In [325]:
! pip install git+https://github.com/anntzer/mplcursors

  Cloning https://github.com/anntzer/mplcursors to /tmp/pip-req-build-6ph2nl5i
  Running setup.py bdist_wheel for mplcursors ... - done
  Stored in directory: /tmp/pip-ephem-wheel-cache-y57sqkhv/wheels/96/31/13/6a044169fec7a9aede6380379640d7ecff720c5dc8606e65bf
Successfully built mplcursors


In [0]:
#import libraries here
%matplotlib inline
import mplcursors
from mpl_toolkits.basemap import Basemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [0]:
#Eliminate ... in panda outputpd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [0]:

#Import Raw Data
CA_Water_Path = 'https://data.cnra.ca.gov/dataset/3f96977e-2597-4baa-8c9b-c433cea0685e/resource/a9e7ef50-54c3-4031-8e44-aa46f3c660fe/download/lab-results.csv'
df = pd.read_csv(CA_Water_Path)

In [0]:
#View Raw Data
df.head(500)

In [0]:
#Separate out surface water information
sw = df[df['STATION_TYPE'] == 'Surface Water']
#check Data types
sw.dtypes

In [0]:
#View data parameters
sw.describe()

In [0]:
#Convert SAMPLE_DATE Col from string to time value 
sw['SAMPLE_DATE'] = pd.to_datetime(sw['SAMPLE_DATE'], infer_datetime_format = True)
#Convert SAMPLE_DATE Col to year only (converted back to int?... leave as int)
sw['SAMPLE_DATE'] = sw['SAMPLE_DATE'].dt.year
sw.head()

In [0]:
#Verify data types
sw.dtypes

In [0]:
#Remove unecessary columns
sw.drop(columns=['STATION_NAME', 'STATION_TYPE', 'STATION_NUMBER', 'SAMPLE_CODE', 'SAMPLE_DEPTH', 'SAMPLE_DEPTH_UNITS', 'METHOD_NAME'], inplace = True)
sw.head()

In [0]:
#Check for missing values
sw['COUNTY_NAME'].isnull().sum()

In [0]:
#Check number of unique values in each column... keep long and lat for possible map display
sw.nunique() 

In [0]:
#Count historical data per county
pd.value_counts(df['COUNTY_NAME'])

In [0]:
#list total number of measurements for each water parameter
pd.value_counts(df['PARAMETER'])

In [0]:
#Check size of df
sw.shape

In [0]:
#Test dynamic date range... most recent to 30 years back
list(range(max(sw['SAMPLE_DATE'])-30, max(sw['SAMPLE_DATE'])))

In [0]:
#Select and separate target counties based on size and geographic location in state
Counties = ('Los Angeles', 'San Diego', 'Sacramento', 'Fresno', 'San Francisco')
#Select and separate data based on dynamic 30 year range
Years = list(range(max(sw['SAMPLE_DATE'])-30, max(sw['SAMPLE_DATE'])))
#Select and separate contaminents based on significance and data availability
Parameters = ('pH', 'Turbidity', 'Total Dissolved Solids')
#Create and view new project data
county_data = (sw.loc[sw['COUNTY_NAME'].isin(Counties)])
year_data = (county_data.loc[county_data['SAMPLE_DATE'].isin(Years)])
target_data = (year_data.loc[year_data['PARAMETER'].isin(Parameters)])
target_data



In [0]:
LL = sw[['FULL_STATION_NAME', 'LONGITUDE', 'LATITUDE']]
LL = LL.drop_duplicates()
LL = LL.replace(np.nan, 'No Name', regex=True)
LL.isnull().sum()

In [0]:
LL

In [0]:
#assess size of new data
LL.shape

In [0]:
#Observe amount of data for each contaminent
pd.value_counts(target_data['PARAMETER']).sort_index()

In [0]:
state_data = pd.crosstab(target_data['SAMPLE_DATE'], target_data['PARAMETER'], values = target_data['RESULT'], aggfunc=[np.mean])
state_data.columns = ['Total Dissolved Solids', 'Turbidity', 'pH']
state_data['Total Dissolved Solids'] = (state_data['Total Dissolved Solids']/max(state_data['Total Dissolved Solids']))*100
state_data['Turbidity'] = (state_data['Turbidity']/max(state_data['Turbidity']))*100
state_data['pH'] = 100 - (((state_data['pH']-min(state_data['pH']))/min(state_data['pH']))*100)

state_data


In [0]:
fig = plt.figure(figsize = (20,5))
plt.plot(state_data);

In [0]:
Fresno_data = (target_data.loc[target_data['COUNTY_NAME'] =='Fresno'])
Fresno_data = pd.crosstab(Fresno_data['SAMPLE_DATE'], Fresno_data['PARAMETER'], values = Fresno_data['RESULT'], aggfunc=[np.mean])
Fresno_data.columns = ['Total Dissolved Solids', 'Turbidity', 'pH']
Fresno_data['Total Dissolved Solids'] = (Fresno_data['Total Dissolved Solids']/max(Fresno_data['Total Dissolved Solids']))*100
Fresno_data['Turbidity'] = (Fresno_data['Turbidity']/max(Fresno_data['Turbidity']))*100
Fresno_data['pH'] = 100 - (((Fresno_data['pH']-min(Fresno_data['pH']))/min(Fresno_data['pH']))*100)

Fresno_data


In [0]:

LA_data = (target_data.loc[target_data['COUNTY_NAME'] =='Los Angeles'])
LA_data = pd.crosstab(LA_data['SAMPLE_DATE'], LA_data['PARAMETER'], values = LA_data['RESULT'], aggfunc=[np.mean])
LA_data.columns = ['Total Dissolved Solids', 'Turbidity', 'pH']
LA_data['Total Dissolved Solids'] = (LA_data['Total Dissolved Solids']/max(LA_data['Total Dissolved Solids']))*100
LA_data['Turbidity'] = (LA_data['Turbidity']/max(LA_data['Turbidity']))*100
LA_data['pH'] = 100 - (((LA_data['pH']-min(LA_data['pH']))/min(LA_data['pH']))*100)

LA_data

In [0]:
Sac_data = (target_data.loc[target_data['COUNTY_NAME'] =='Sacramento'])
Sac_data = pd.crosstab(Sac_data['SAMPLE_DATE'], Sac_data['PARAMETER'], values = Sac_data['RESULT'], aggfunc=[np.mean])
Sac_data.columns = ['Total Dissolved Solids', 'Turbidity', 'pH']
Sac_data['Total Dissolved Solids'] = (Sac_data['Total Dissolved Solids']/max(Sac_data['Total Dissolved Solids']))*100
Sac_data['Turbidity'] = (Sac_data['Turbidity']/max(Sac_data['Turbidity']))*100
Sac_data['pH'] = 100 - (((Sac_data['pH']-min(Sac_data['pH']))/min(Sac_data['pH']))*100)

Sac_data

In [0]:
SD_data = (target_data.loc[target_data['COUNTY_NAME'] =='San Diego'])
SD_data = pd.crosstab(SD_data['SAMPLE_DATE'], SD_data['PARAMETER'], values = SD_data['RESULT'], aggfunc=[np.mean])
SD_data.columns = ['Total Dissolved Solids', 'Turbidity', 'pH']
SD_data['Total Dissolved Solids'] = (SD_data['Total Dissolved Solids']/max(SD_data['Total Dissolved Solids']))*100
SD_data['Turbidity'] = (SD_data['Turbidity']/max(SD_data['Turbidity']))*100
SD_data['pH'] = 100 - (((SD_data['pH']-min(SD_data['pH']))/min(SD_data['pH']))*100)

SD_data

In [0]:
lat = LL['LATITUDE'].values
lon = LL['LONGITUDE'].values
fig = plt.figure(figsize=(16, 16))
m = Basemap(projection='lcc', resolution='h', 
            lat_0=37.5, lon_0=-119,
            width=1E6, height=1.2E6)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='black')
m.drawstates(color='black')
m.scatter(lon, lat, latlon= True, s= 12);



In [0]:
import altair as alt
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')
capitals = data.us_state_capitals.url

# US states background
background = alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).properties(
    title='US State Capitols',
    width=700,
    height=400
).project('albersUsa')


base = alt.Chart(capitals).encode(
    longitude='lon:Q',
    latitude='lat:Q'
)

text = base.mark_text(dy=-5, align='right').encode(
    alt.Text('city', type='nominal'),
    opacity=alt.condition(~hover, alt.value(0), alt.value(1))
)

